In [4]:
import glob
from pathlib import Path
import numpy as np
import vtk
from vtk.util.numpy_support import vtk_to_numpy

import matplotlib.pyplot as plt

# Locate the first .vti file
vti_files = sorted(Path("out/").glob("*.vti"))
reader = vtk.vtkXMLImageDataReader()
if not vti_files:
    raise FileNotFoundError("No .vti files found in the current directory.")

for vti_file in vti_files:
    # Read the VTI data

    reader.SetFileName(str(vti_file))
    reader.Update()
    image_data = reader.GetOutput()

    # Extract velocity data (uses the first array in point data by default) 
    point_data = image_data.GetPointData()
    if point_data.GetNumberOfArrays() == 0:
        raise ValueError("No point data arrays found in the VTI file.")

    uArray = point_data.GetArray(0)
    vArray = point_data.GetArray(1)
    num_components = uArray.GetNumberOfComponents()

    # Convert to NumPy and reshape to (z, y, x, components)
    dims = image_data.GetDimensions()  # (nx, ny, nz)
    u_np = vtk_to_numpy(uArray).reshape((dims[2], dims[1], dims[0], num_components))
    v_np = vtk_to_numpy(vArray).reshape((dims[2], dims[1], dims[0], num_components))
    print(u_np.shape)

    # Compute magnitude if vector, otherwise use scalar values
    velocity_mag = (
        np.linalg.norm(velocity_np, axis=-1) if num_components > 1 else velocity_np[..., 0]
    )

# Plot a mid-z slice of the velocity magnitude
mid_slice = velocity_mag[velocity_mag.shape[0] // 2, :, :]

plt.figure(figsize=(6, 5))
plt.imshow(mid_slice, origin="lower", cmap="viridis")
plt.colorbar(label="Velocity magnitude")
plt.title(f"Velocity slice from {first_vti.name}")
plt.xlabel("X index")
plt.ylabel("Y index")
plt.tight_layout()
plt.show()

ValueError: cannot reshape array of size 1323 into shape (1,21,21,1)